In [0]:
%python
tenant_id = ""
client_id = ""
client_secret = ""
storage_account_name = "azuredeproject1603"

configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": client_id,
    "fs.azure.account.oauth2.client.secret": client_secret,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

container_name = "tokyo-olympics-data"
mount_point = "/mnt/tokyomount"

dbutils.fs.mount(
    source=f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point=mount_point,
    extra_configs=configs
)

# Verify the connection
display(dbutils.fs.ls(mount_point))

path,name,size,modificationTime
dbfs:/mnt/tokyomount/raw-data/,raw-data/,0,1734690469000
dbfs:/mnt/tokyomount/transformed-data/,transformed-data/,0,1734690478000


In [0]:
%fs
ls "/mnt/rawdata"

path,name,size,modificationTime
dbfs:/mnt/rawdata/tokyo-data/,tokyo-data/,0,1734684220000


In [0]:
coaches = spark.read.format("csv").load(f"{mount_point}/raw-data/coaches.csv", header=True)
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [0]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
from pyspark.sql.functions import when

coaches = coaches.withColumn(
    "Event", when(coaches["Event"] == "Men", "M").when(coaches["Event"] == "Women", "F")
)

coaches.show()

+--------------------+--------------------+-----------------+-----+
|                Name|             Country|       Discipline|Event|
+--------------------+--------------------+-----------------+-----+
|     ABDELMAGID Wael|               Egypt|         Football| NULL|
|           ABE Junya|               Japan|       Volleyball| NULL|
|       ABE Katsuhiko|               Japan|       Basketball| NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football| NULL|
|          AGEBA Yuya|               Japan|       Volleyball| NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|    M|
|       AL SAADI Kais|             Germany|           Hockey|    M|
|       ALAMEDA Lonni|              Canada|Baseball/Softball| NULL|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|    M|
|     ALEKSEEV Alexey|                 ROC|         Handball|    F|
|ALLER CARBALLO Ma...|               Spain|       Basketball| NULL|
|       ALSHEHRI Saad|        Saudi Arabia|     

In [0]:
medals = spark.read.format("csv").load(f"{mount_point}/raw-data/Medals.csv", header=True)
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
from pyspark.sql.types import IntegerType

medals = medals.withColumn("Rank", medals["Rank"].cast(IntegerType())) \
                .withColumn("Gold", medals["Gold"].cast(IntegerType())) \
                .withColumn("Silver", medals["Silver"].cast(IntegerType())) \
                .withColumn("Bronze", medals["Bronze"].cast(IntegerType()))\
                .withColumn("Total", medals["Total"].cast(IntegerType())) \
                .withColumn("Rank by Total", medals["Rank by Total"].cast(IntegerType()))


In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
coaches.write.mode("overwrite").option("header","true").csv("/mnt/tokyomount/transformed-data/coaches.csv")
medals.write.mode("overwrite").option("header","true").csv("/mnt/tokyomount/transformed-data/medals.csv")

In [0]:
dbutils.fs.unmount("/mnt/rawdata")

/mnt/rawdata has been unmounted.


True